In [119]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device used: {device.type}')

torch.cuda.is_available()

Device used: cpu


False

# Hyperparameters for project

In [120]:
train_size = 0.7
test_size = 1 - train_size
num_epochs = 5
batch_size = 4
learning_rate = 0.001

# Data preprocessing


## Import the dataset
Dataset should be imported as a pytorch dataloader for batch optimization

Custom Dataset

## **Data augmentation**

To augment the data before training, we will attempt to use two methods:


1.   Scaling
> Scaling is used because we wish to taken into account the varying structure size of tumors and skulls in the images.

2.   Noise Injection
> Noise injection is used to help assist the model in learning the complex patterns around the tumors and make it more robust to small changes in the data.
> We will experiment with both Gaussian (random), and salt-and-paper (random values to min. or max. values, 0 to 255) noise injection.

All combinations of these will be used to determine their effectiveness, and if they introduce any *bias*, *artifacts*, or *overfitting* both in isolation, or combination.

The order of the data augmentation will be:
1.   No Data Augmentation
2.   Scaling
3.   Noise Injection
4.   Scaling, Noise Injection

In [121]:
from torchvision.transforms import v2

def add_noise_gaussian(tensor, mean = 0, std = 0.05):
    """
    Parameters:
    - tensor: PyTorch tensor data type without noise (input)
    - mean: Mean of the Gaussian distribution
    - std: Standard deviation of the Gaussian distribution

    Returns:
    - tensor + noise: PyTorch tensor data type with noise (output)
    """
    noise = torch.randn(tensor.size()) * std + mean
    return tensor + noise

def add_noise_salt_pepper(tensor, salt_prob = 0.02, pepper_prob = 0.02):
    """
    Parameters:
    - tensor: PyTorch tensor data type without noise (input)
    - salt_prob: Probability that salt noise is added (full white)
    - pepper_prob: Probability that pepper noise is added (full black)

    Returns:
    - tensor + salt_mask = pepper_mask: PyTorch tensor data type with noise (output)
    that ensured to be between 0 and 1
    """

    salt_mask = (torch.rand_like(tensor) < salt_prob).float()
    pepper_mask = (torch.rand_like(tensor) < pepper_prob).float()

    return torch.clamp((tensor + salt_mask - pepper_mask), 0, 1)

# Abitrary values (set to double of base image height*width)
resize_x = 256
resize_y = 256

# Set resize_x and resize_y before using these transforms
# All transforms convert it to a tensor with the dimensions of (Channels, Height, Width)
transforms = {
    'none': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True)
                        ]),
    'scale': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Resize((resize_x, resize_y), antialias=True)
                        ]),
    'noise_gaussian': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Lambda(lambda x: add_noise_gaussian(x))
                        ]),
    'noise_salt_pepper': v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Lambda(lambda x: add_noise_salt_pepper(x))
                        ]),
    'all_gaussian':     v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Resize((resize_x, resize_y), antialias=True),
                        v2.Lambda(lambda x: add_noise_gaussian(x))
                        ]),
    'all_salt_pepper':  v2.Compose([
                        v2.ToImage(), 
                        v2.ToDtype(torch.float32, scale=True),
                        v2.Resize((resize_x, resize_y), antialias=True),
                        v2.Lambda(lambda x: add_noise_salt_pepper(x))
                        ])
}

In [122]:
from torch.utils.data import Dataset
from torchvision.transforms import v2
from PIL import Image
import os

class CustomDataset(Dataset):

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.class_folders = os.listdir(root_dir)
        self.class_to_idx = {class_folder: i for i, class_folder in enumerate(self.class_folders)}
        self.images = self.make_dataset()

    def make_dataset(self):
        images = []
        for class_folder in self.class_folders:
            class_path = os.path.join(self.root_dir, class_folder)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                images.append((img_path, self.class_to_idx[class_folder]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path)

        random_number = np.random.uniform(0, 1)

        # if self.transform:
        #     image = self.apply_random_transform(image, random_number)
        image = transforms['none'](image)

        return image, label
    
    def apply_random_transform(self, image, random_number):
        # Example: Apply different transformations based on the random number
        if random_number < 0.25:
            return transforms['noise_gaussian'](image)
        elif random_number < 0.5:
            return transforms['noise_salt_pepper'](image)
        else:
            return transforms['none'](image)
    

In [123]:
from torch.utils.data import DataLoader

transform = v2.Compose([
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True)
])

dataset = CustomDataset(root_dir='Dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [124]:
from sklearn.model_selection import train_test_split

train_size = 0.8  # You can adjust the ratio based on your needs
test_size = 1 - train_size
batch_size = 16    # Adjust according to your needs

# Assuming your dataset is a list or any data structure that can be split
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)

# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Data normalization

# Architecture of the network

**CNN Model Creation**

In [125]:
class CustomCNN(nn.Module):
    def __init__(self, weights="DEFAULT"):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # MRI images are grayscale, so in_channels=1
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)  # Adjust for the flattened conv3 output
        self.fc2 = nn.Linear(256, 4)  # 4 classes in our dataset

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [126]:
model = CustomCNN()

model

CustomCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=32768, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=4, bias=True)
)

In [127]:
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR

def select_optimizer(optimizer_name, parameters, lr=1e-3, weight_decay=0.):
    if optimizer_name == "sgd":
        return torch.optim.SGD(parameters, lr=lr, weight_decay=weight_decay, momentum=0.9)
    elif optimizer_name == "rmsprop":
        return torch.optim.RMSprop(parameters, lr=lr, weight_decay=weight_decay, alpha=0.99)
    elif optimizer_name == "adam":
        return torch.optim.Adam(parameters, lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")


# Choose optimizer and regularization hyperparameters
optimizer_name = "adam"   # Could be "sgd", "rmsprop", or "adam"
learning_rate = 0.001
weight_decay = 0.0

optimizer = select_optimizer(optimizer_name=optimizer_name, parameters=model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [128]:
def train(model, device, train_loader, optimizer, scheduler, epoch):
    # Set the model in "training" mode, enabling features like dropout and
    # batch normalization that are specific to training.
    model.train()

    # We iterate over `train_loader`, which batches the training data.
    # `enumerate(train_loader)` gives us a counter `batch_idx` and the data in data and target labels in target.
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move our data and labels to the device we are using (CPU or GPU),
        # enabling accelerated computation.
        data, target = data.to(device), target.to(device)

        # Clear old gradients; if not cleared, they would accumulate with subsequent backward passes.
        optimizer.zero_grad()

        output = model(data)  # Forward pass to get predictions.
        loss = F.cross_entropy(output, target)
        loss.backward()       # Backpropagation to compute the gradients.

        # Update the weights of the model based on the gradients calculated during backpropagation.
        optimizer.step()

        if batch_idx % 100 == 0:
            current_lr = scheduler.get_last_lr()[0]  # Access the last learning rate
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} "
                  f"({100. * batch_idx / len(train_loader):.0f}%)]\t"
                  f"Loss: {loss.item():.6f} (LR: {current_lr:.6f})")

    # Update the learning rate according to the specified schedule
    scheduler.step()

In [129]:
def test(model, device, test_loader):
    # Set the model to "evaluation" mode. This is necessary because certain layers
    # like dropout layers behave differently during training than during testing.
    model.eval()

    test_loss = 0
    correct = 0

    # Context manager under which all the operations will have `requires_grad=False`,
    # meaning that PyTorch will not calculate or keep track of gradients.
    # This is used because gradient computation is not needed for evaluation and saves memory and computation.
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # The loss is summed up across all batches.
            # The reduction="sum" parameter ensures that the losses are added together.
            test_loss += F.cross_entropy(output, target, reduction="sum").item()

            # Get the index of the max log-probability
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

    # Total loss is divided by the number of items in the dataset to get the average loss.
    test_loss /= len(test_loader.dataset)
    # Accuracy is calculated as the percentage of correct predictions over the total number of predictions.
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n")

In [130]:
epochs = 5

# Determine if a GPU with CUDA support is available and use it; otherwise, use the CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using {device} device.\n")

# Begin the training process
for epoch in range(1, epochs + 1):
    # Train the model for one epoch: go through all batches in the training dataset.
    train(model, device, train_loader, optimizer, scheduler, epoch)

    # After each epoch, evaluate the model on the test dataset to monitor its performance.
    test(model, device, test_loader)

Using cpu device.

Train Epoch: 1 [0/5120 (0%)]	Loss: 1.400377 (LR: 0.001000)
Train Epoch: 1 [1600/5120 (31%)]	Loss: 0.841064 (LR: 0.001000)
Train Epoch: 1 [3200/5120 (62%)]	Loss: 1.227140 (LR: 0.001000)
Train Epoch: 1 [4800/5120 (94%)]	Loss: 1.107511 (LR: 0.001000)

Test set: Average loss: 0.9549, Accuracy: 696/1280 (54%)

Train Epoch: 2 [0/5120 (0%)]	Loss: 1.086693 (LR: 0.000700)
Train Epoch: 2 [1600/5120 (31%)]	Loss: 0.731400 (LR: 0.000700)
Train Epoch: 2 [3200/5120 (62%)]	Loss: 0.897253 (LR: 0.000700)
Train Epoch: 2 [4800/5120 (94%)]	Loss: 0.661364 (LR: 0.000700)

Test set: Average loss: 0.7689, Accuracy: 820/1280 (64%)

Train Epoch: 3 [0/5120 (0%)]	Loss: 0.682647 (LR: 0.000490)
Train Epoch: 3 [1600/5120 (31%)]	Loss: 0.450650 (LR: 0.000490)
Train Epoch: 3 [3200/5120 (62%)]	Loss: 0.427601 (LR: 0.000490)
Train Epoch: 3 [4800/5120 (94%)]	Loss: 0.743272 (LR: 0.000490)

Test set: Average loss: 0.6579, Accuracy: 922/1280 (72%)

Train Epoch: 4 [0/5120 (0%)]	Loss: 0.844691 (LR: 0.000343)
T